## Introduction

The dataset was provided by a company in the chocolate marketing sector with international operations. It contains information from the period 2022–2024 and integrates variables such as sales force, markets (countries), product portfolio, transaction date, revenue generated (amount), and volume distributed, measured in the number of boxes shipped.

## Preparation

In [ ]:
# Import libraries
import pandas as pd
import numpy as ny
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

In [ ]:
# Download Dataset
path = r"C:/Users/Pablo/Documents/Data_projects/2601_Chocolate_sales/Chocolate Sales (2).csv"
df = pd.read_csv(path)

In [ ]:
# Visualize dataframe information
print(df.info())

In [ ]:
# Show dataframe shape
print(f"Loaded Dataset: {df.shape[0]} rows, {df.shape[1]} columns")

In [ ]:
# Show dataframe
print(df)

Note:
- The column header format is incorrect.
- The date and amount columns do not have the correct data type.
- No missing values ​​are observed.
- No Boolean values ​​are observed.

## Data cleaning

In [ ]:
# ========================
# Cleaning headers
# ========================
print(df.columns.tolist())

df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')
print("Fix applied")
print(df.columns.tolist())

In [ ]:
# ===================================
# Type conversion & currency cleaning
# ===================================

# 1. Define a mask for the amount column
# 2. Convert the data type to a string
# 3. Return the values ​​that contain the $ symbol within the string
dirty_amount_mask = df['amount'].astype(str).str.contains(r'\$')
print(df.loc[dirty_amount_mask, ['product','amount']].head(3))

# 1. Select the amount column
# 2. Convert the data type to a string
# 3. Replace any non-digit characters "." or "-" with nothing to ensure you are working only with numbers
df['amount'] = df['amount'].astype(str).str.replace(r'[^\d.-]', '', regex=True)
# 4. Convert the strings in the "amount" column back to numbers
df['amount'] = pd.to_numeric(df['amount'])

print()
print("Fix applied")
print(df.loc[dirty_amount_mask, ['product','amount']].head(3))

In [ ]:
# =====================================
# Categorial typos
# =====================================

# Verify that there are no duplicates in the categorical columns
print(df['sales_person'].unique())
print()
print(df['country'].unique())
print()
print(df['product'].unique())

In [ ]:
# =====================================
# Date parsing
# =====================================

# Check the type
print(df['date'].dtype)

# Change of column type for date
df['date'] = pd.to_datetime(df['date'], dayfirst=True,errors='coerce')

print()
print("Fix applied")
print(df['date'].dtype)


In [ ]:
# =============================
# Handling outliers
# =============================

Q1 = df['amount'].quantile(0.25)
Q3 = df['amount'].quantile(0.75)

# Inter quantile range
IQR = Q3 - Q1
upper_limit = Q3 + (3 * IQR)

outlier_mask = df['amount'] > upper_limit
print(df.loc[outlier_mask, ['product', 'amount']].head(3))

print()
print("Fix applied")
df.loc[outlier_mask, 'amount'] = upper_limit
print(df.loc[outlier_mask, ['product', 'amount']].head(3))


In [ ]:
# Verify that no information was lost
print(df.info())

In [ ]:
# Extract the fragmented date into new columns
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [ ]:
# Define the quarter of operations
def quarter(month):
    if month in [1,2,3]:
        return 'Q1'
    elif month in [4,5,6]:
        return 'Q2'
    elif month in [7,8,9]:
        return 'Q3'
    else:
        return 'Q4'

df['quarter'] = df['month'].apply(quarter)

In [ ]:
# Get the unit price per operation
df['amount_per_unit'] = df['amount'] / df['boxes_shipped']

In [ ]:
# Visualize the dataframe with new columns
df

## Exploratory Data Analysis (EDA)

### Income

In [ ]:
# Show sales and percentage by country
country_sales = df.groupby(['country'])['amount'].sum().sort_values(ascending=False)
country_percentage = (country_sales / country_sales.sum() * 100).sort_values(ascending=False)

print(country_sales, country_percentage)

In [ ]:
# Show sales by country
order = df.groupby(['country'])['amount'].sum().sort_values(ascending=False).index

plt.figure(figsize=(10,5))

sns.barplot(
    data=df,
    x="country",
    y="amount",
    estimator=sum,  # suma ventas por país
    palette="mako",
    order=order
)

plt.title("Sales by country")
plt.xticks(rotation=45)
plt.show()

Note:
- Revenues are well diversified by country, with each market contributing a similar share.
- Australia contributes the most revenue at 18.42%, while New Zealand contributes the least at 15.37%.

In [ ]:
# Show sales by product
product_sales = df.groupby(['product'])['amount'].sum().sort_values(ascending=False)
product_percentage = (product_sales / product_sales.sum() * 100).sort_values(ascending=False)

print(product_sales)

In [ ]:
# Show product share
print(product_percentage)

In [ ]:
# Show sales by product
order = df.groupby(['product'])['amount'].sum().sort_values(ascending=False).index

plt.figure(figsize=(10,5))

sns.barplot(
    data=df,
    x="product",
    y="amount",
    estimator=sum,  # sum sales by product
    palette="mako",
    order=order
)

plt.title("Sales by product")
plt.xticks(rotation=90)
plt.show()

Note:
- Product revenue is well diversified; no single product stands out from the rest.
- The product that contributes the most to revenue is Smooth Silky Salty at 5.65%, while the one that contributes the least is 70% Dark Bites at 3.42%.

### Operation

In [ ]:
# Count sales by country
print(df['country'].value_counts(ascending=False))

In [ ]:
# Plot count sales by country
order = df['country'].value_counts(ascending=False).index

sns.countplot(
    data=df, 
    x="country",
    palette="mako",
    order=order
)

plt.title("Sales count by country")
plt.show()

Note:
- Sales are well distributed across the different markets, with Australia standing out slightly.
- The highest number of sales is in Australia, the lowest in New Zealand.

In [ ]:
# Price per unit
amount_per_unit = df.groupby(['country'])['amount_per_unit'].mean().sort_values(ascending=False)

print(amount_per_unit)

Note:
- Canada is the most profitable market per unit, while the UK is the least.

In [ ]:
# Volume per country
volume_per_country = df.groupby(['country'])['boxes_shipped'].sum().sort_values(ascending=False)
Volume_per_country_percentage = volume_per_country / volume_per_country.sum() * 100

print(volume_per_country)
print(Volume_per_country_percentage)

In [ ]:
# Show volume by country
order = df.groupby(['country'])['boxes_shipped'].sum().sort_values(ascending=False).index

plt.figure(figsize=(10,5))

sns.barplot(
    data=df,
    x="country",
    y="boxes_shipped",
    estimator=sum,  # sum boxes shipped by country
    palette="mako",
    order=order
)

plt.title("Volume by country")
plt.xticks(rotation=90)
plt.show()

Note:
- The volume of boxes shipped to each market is fairly balanced.
- The Australian market receives the most boxes, while New Zealand receives the fewest.

### Salesforce

In [ ]:
# Sales per seller
sales_seller = df.groupby('sales_person')['amount'].sum().sort_values(ascending=False)
# % per seller
sales_seller_percentage = sales_seller / sales_seller.sum() * 100

print(sales_seller)

In [ ]:
# % per seller
print(sales_seller_percentage)

In [ ]:
# Show sales by seller
order = df.groupby('sales_person')['amount'].sum().sort_values(ascending=False).index

plt.figure(figsize=(10,5))

sns.barplot(
    data=df,
    x="sales_person",
    y="amount",
    estimator=sum,  # sum sales by seller
    palette="mako",
    order=order
)

plt.title("Sales by seller")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Sales performance per year
sales_year_sp = df.groupby(["year", "sales_person"])["amount"].sum().reset_index()

plt.figure(figsize=(12,6))

sns.lineplot(
    data=sales_year_sp,
    x="year",
    y="amount",
    hue="sales_person",
    marker="o"
)

plt.title("Sales by Sales person Over Time")
plt.ylabel("Total Sales")
plt.xlabel("Year")
plt.show()

In [ ]:
# Create a dataframe to evaluate sales performance
sp_perf = df.groupby("sales_person").agg(
    total_sales=("amount", "sum"),
    avg_sale=("amount", "mean"),
    transactions=("amount", "count"),
    total_boxes=("boxes_shipped", "sum")
).reset_index()

print(sp_perf)

In [ ]:
# It tells you how far each salesperson is from the team average.
sp_perf["z_sales"] = (sp_perf["total_sales"] - sp_perf["total_sales"].mean()) / sp_perf["total_sales"].std()
print(sp_perf)

In [ ]:
# Define function to classify sellers
def classify(z):
    if z > 1:
        return "High Performer"
    elif z < -1:
        return "Low Performer"
    else:
        return "Average"

sp_perf["performance"] = sp_perf["z_sales"].apply(classify)
print(sp_perf[['sales_person','performance']])

In [ ]:
# Create the percentile column
sp_perf["percentile"] = sp_perf["total_sales"].rank(pct=True)

In [ ]:
# Classify sales person according to percentiles
sp_perf["segment"] = pd.cut(
    sp_perf["percentile"],
    bins=[0, 0.4, 0.8, 1],
    labels=["Low", "Medium", "Top"]
)
print(sp_perf[sp_perf['segment'] == 'Low'][['sales_person', 'segment']])
print()
print(sp_perf[sp_perf['segment'] == 'Top'][['sales_person', 'segment']])

Note
- Regarding the salespeople, differences were observed in their group performance.
- Salespeople were ranked based on the group's average performance and also according to their percentile of total sales.
- Two lists of salespeople were generated: high performers to receive some type of benefit, and low performers to investigate why they were below average.

### Time

In [ ]:
# Sellers by year
# Set the date as the index
df.set_index(keys='date', inplace=True)

In [ ]:
# Shows sales by year
print(df.groupby('year')['amount'].sum())

In [ ]:
# Annual sales visualization
year_sales= df['amount'].resample('1YE').sum().reset_index()

plt.figure(figsize=(15,6))

sns.lineplot(
    data=year_sales,
    x='date',
    y='amount',
    marker='o'
)

plt.title('Yearly Sales')
plt.show()

In [ ]:
# Shows annual sales by quarter
print(df.groupby(['year', 'quarter'])['amount'].sum())

In [ ]:
# Visualize annual sales by quarter
df.groupby(['year', 'quarter'])['amount'].sum().plot(figsize=(15,5), title='Amount by Quarter 2022-2024') 
plt.xlabel('date') 
plt.ylabel('amount') 
plt.show()

In [ ]:
# Shows monthly sales
print(df.groupby(['year','month'])['amount'].sum())

In [ ]:
# Monthly sales visualization
df.resample('1ME')['amount'].sum().plot(figsize=(15,5), title='Monthly Amount 2022-2024')
plt.xlabel('date')
plt.ylabel('amount')
plt.show()

Note:
- A growth of 7.43% was observed from 2022 to 2023 and 4.83% from 2023 to 2024.
- When looking at sales by quarter, it is clear that there are no sales in the last quarter of each year, indicating a seasonal nature of the business.
- When looking at sales by month, it is observed that there are no sales from September until the end of the year, making it a good time to build up inventory for the following season.

## Conclusions

The analysis of the 2022–2024 sales data reveals a business characterized by diversification, operational balance, and predictable seasonality.

Revenue distribution across countries is relatively even, reducing geographic concentration risk and reflecting a stable international presence. Although Australia leads in revenue share (18.42%), sales volume, and boxes shipped, the differences between markets are moderate. New Zealand contributes the least (15.37%), but not at a level that represents structural weakness. Overall, the company does not depend excessively on any single market.

The product portfolio shows the same pattern. Revenue contribution is well distributed, with no product representing a critical dependency. The highest contributor, Smooth Silky Salty (5.65%), and the lowest, 70% Dark Bites (3.42%), remain within a narrow range. This indicates a balanced commercial strategy, lowering risk related to demand shifts in specific products.

Operationally, the number of boxes shipped is also evenly distributed across markets, consistent with revenue patterns. This suggests pricing stability and healthy alignment between volume and income generation.

From a performance standpoint, sales growth is positive but shows a slight deceleration: revenues increased by 7.43% from 2022 to 2023 and by 4.83% from 2023 to 2024. While the company continues to grow, the slowdown may signal market maturation or the need for renewed commercial strategies to sustain higher growth rates.

A key structural characteristic of the business is its seasonal behavior. No sales are recorded in the last quarter of each year, and monthly analysis shows activity stopping from September onward. This predictable seasonality provides an operational advantage: the off-season period can be strategically used for inventory buildup, logistics planning, maintenance, and salesforce training ahead of the next commercial cycle.

Greater variability appears at the salesforce level. Differences in individual performance highlight opportunities for targeted management actions. High-performing salespeople can be supported through incentive and retention programs, while lower-performing individuals warrant further analysis to identify training needs, market allocation issues, or process inefficiencies. Therefore, while the company demonstrates structural stability at the market and product levels, salesforce optimization represents the primary opportunity for performance improvement.

In summary, the company operates with a low-risk, diversified commercial structure, balanced demand across markets and products, and a clear seasonal pattern that enables operational planning. Future strategy should focus on maintaining diversification, leveraging strong markets such as Australia, addressing the recent slowdown in growth, and improving salesforce effectiveness to unlock additional performance gains.